In [1]:
import sys
# sys.path.append("D:\thesis\thesis\rllib")

import rllib as rl
import json
import joblib
import numpy as np
import torch

dataset_train = "./data_preprossing/output/train_cleaned.csv"
dataset_test = "./data_preprossing/output/test_cleaned.csv"
symptom_nlice_file ="./data/basic/symptom_nlice.json"
# symptom_map_file = "./data/basic/symptoms_db.json"
symptom_map_file = "./data_preprossing/output/basic/symptom_db_v1.json"
condition_map_file = "./data/basic/conditions_db.json"
body_parts="./data/basic//body-parts-enc.json"
excitation_enc="./data/basic/excitation_encoding.json"
frequency_enc="./data/basic/frequency_encoding.json"
nature_enc="./data/basic/nature_encoding.json"
vas_enc="./data/basic/vas_encoding.json"
onset_enc="./data/basic/onset_encoding.json"
duration_enc="./data/basic/duration_encoding.json"
import datetime
import sys
from pathlib import Path
curr_path = str(Path().absolute())
parent_path = str(Path().absolute().parent)
sys.path.append(parent_path) # add current terminal path to sys.path
curr_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # obtain current time
test = "./data_preprossing/output/data/total_1k.csv"
test_v1 = "./data_preprossing/output/data/total_50.csv"
# with open(symptom_map_file) as fp:
#             symptoms = json.load(fp)
#             sorted_symptoms = sorted(symptoms.keys())
#             symptom_map = {code: idx for idx, code in enumerate(sorted_symptoms)}
# with open(symptom_map_file, "w") as fp:
#     json.dump(symptom_map, fp, indent=4)
piors = "./data/basic/all_piors.json"

In [2]:
env = rl.environment.RLBasicMedEnvContext(
    data_file=test,
    symptom_map_file=symptom_map_file,
    condition_map_file=condition_map_file,
    body_parts=body_parts,
    excitation_enc=excitation_enc,
    frequency_enc=frequency_enc,
    nature_enc=nature_enc, 
    vas_enc=vas_enc, 
    onset_enc=onset_enc, 
    duration_enc=duration_enc,
    piors = piors,
    classifer="nb",
    max_turn=40,
    policy = False,
    epoch =33000
)

In [3]:
input_dim = env.num_symptoms*8
output_dim = env.num_symptoms + 33
print(env.num_symptoms)
layer_config = [
    [input_dim, 128],
    [128, 64],
    [64, 48],
    [48, 32],
    [32, 32],
    [32, 48],
    [48, 64],
    [64, output_dim]
]

learning_start = 1280
batch_size = 64
target_update = 1280
replay_capacity = 100000

61


In [4]:
agent = rl.agent.MedAgent(
    env,
    layer_config=layer_config,
    learning_start=learning_start,
    batch_size=batch_size,
    target_update=target_update,
    replay_capacity=replay_capacity,
    debug=False,
    train = True, 
    input_dim = input_dim,
    output_dim = output_dim,
    context = False
)

In [5]:
agent.debug = False
bench = rl.bench.MedBench(agent,66000)
bench.run_trial(debug=False)
cfg = rl.config.DQNConfig()
rl.common.make_dir(cfg.result_path,cfg.model_path)
agent.save(cfg.model_path)


In [7]:

cfg = rl.config.DQNConfig()
rewards,success,average_rewards,average_steps,loss =bench.rewardList,bench.get_success_rate(),bench.get_average_rewards(),bench.get_average_turn(),bench.get_loss()
print(success)
print(average_rewards,average_steps)

env1 = rl.environment.RLBasicMedEnvContext(
    data_file= "./data_preprossing/output/data/total_1k_min3.csv",
    symptom_map_file=symptom_map_file,
    condition_map_file=condition_map_file,
    symptom_nlice_file=symptom_nlice_file,
    body_parts=body_parts,
    excitation_enc=excitation_enc,
    frequency_enc=frequency_enc,
    nature_enc=nature_enc, 
    vas_enc=vas_enc, 
    onset_enc=onset_enc, 
    duration_enc=duration_enc,
    classifer="nb",
    max_turn=40,
    policy=True,
    piors = piors,
    epoch = 33000
)
# agent.policy_network.load_state_dict(torch.load('./outputs/DQN/20211219-102427/models/dqn_checkpoint_policy.pth'))
agent.env = env1
agent.train = False
agent.eps_start = 0
agent.eps_end = 0
agent.debug = False
agent.frame_idx = 0
testbench = rl.bench.MedBenchEval(agent,33000)
testbench.run_trial(debug=False)
rewards,success,average_rewards,average_steps,hint_rate,top5,top3 =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn(),testbench.get_hint_rate(),testbench.get_top5_rate(),testbench.get_top3_rate()
print(testbench.success)
print(success)
print(average_rewards,average_steps)
print(hint_rate)
print(top5)
print(top3)



 

0.3419090909090909
30.72219696969697 2.291848484848485
12877
0.39021212121212123
43.09290909090909 3.241
0.5070606060606061
0.6071212121212122
0.5298787878787878


In [8]:

agent.env.policy = False
agent.train = False
agent.eps_start = 0
agent.eps_end = 0
agent.debug = False
agent.frame_idx = 0
testbench = rl.bench.MedBenchEval(agent,33000)
testbench.run_trial(debug=False)
rewards,success,average_rewards,average_steps,hint_rate,top5,top3 =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn(),testbench.get_hint_rate(),testbench.get_top5_rate(),testbench.get_top3_rate()
print(testbench.success)
print(success)
print(average_rewards,average_steps)
print(hint_rate)
print(top5)
print(top3)

14263
0.4322121212121212
12.684212121212122 1.8118181818181818
0.18009090909090908
0.5496363636363636
0.49896969696969695


In [6]:
print(env1.conditions_reverse)

{0: 'Acute pancreatitis', 1: 'Acute pyelonefritis', 2: 'Appendicitis', 3: 'Asthma', 4: 'Atrophic gastritis', 5: 'Bacterial meningitis', 6: 'Bladder cancer', 7: 'Breast cancer  female', 8: 'COPD', 9: 'Carpal tunnel syndrome', 10: 'Cauda equina', 11: 'Colon cancer', 12: 'Covid-19', 13: 'Female cystitis', 14: 'Functional constipation', 15: 'Genital herpes  female', 16: 'Genital herpes  male', 17: 'Giardiasis', 18: 'Gonorrhea, feminine', 19: 'Gout', 20: 'Hernia Nuclei Pulposi', 21: 'IBS (Constipation)', 22: 'Iron deficiency anaemia', 23: 'Lower urinary tract infection', 24: 'Lumbago', 25: 'Lumbar spinal stenosis', 26: 'Lung cancer', 27: 'Mastoiditis', 28: 'Migraine', 29: 'Norovirus gastroenteritis', 30: 'Oesophageal carcinoma', 31: 'Osteoarthritis', 32: 'Otitis externa', 33: 'Otitis media acuta', 34: 'Ovarian cancer', 35: 'Pneumococcal pneumonia', 36: 'Pneumothorax', 37: 'Pulmonary edema  old', 38: 'Pulmonary edema  oldest', 39: 'Pulmonary edema  young', 40: 'Rheumatoid arthritis  feminine

In [8]:
default_age={
"age-0-1-years": 1.536246615603,
"age-1-4-years" : 4.6087398468089,
"age-5-14-years" : 11.4611269862763,
"age-15-29-years" : 21.3235812629491,
"age-30-44-years" :20.2551962493181,
"age-45-59-years" : 20.4205176044076,
"age-60-74-years" : 13.987739661922,
"age-g-75-years" : 6.4068517727151
}
default_sex={
    "sex-male":50,
    "sex-female":50,
}
def get_pro_age(condition,age,all_piors):
    value = all_piors.get(condition)
    age_keys = value.get("age")
    age_keys = default_age if age_keys=={} else age_keys
    for a in age_keys.keys():
        ages = a.split("-")
        age_min,age_max = ages[1],ages[2]
        if age_min =="g":
            age_min=age_max
            age_max=140
        age_min,age_max = int(age_min),int(age_max)
        if (age>=age_min and age<=age_max):
            return age_keys[a]
    return 0



def get_pro_sex(condition,sex,all_piors):
    value = all_piors.get(condition)
    sex_keys = value.get("sex")
    sex_keys = default_sex if sex_keys=={} else sex_keys
    # print(sex_keys)
    gender = "sex-male" if sex == '0' else "sex-female"
    return sex_keys[gender]

def get_pro_incidence(condition,all_piors):
    v = all_piors.get(condition)
    value = v.get("incidence")
    value = value.split('/')
    result = float(value[0])/float(value[1])
    return result


a = torch.tensor([[-0.7547, -1.3846, -0.5159,  0.7778, -0.0657, -0.0527,  0.1470, -0.3202,
          0.6473, -0.2663,  0.5621, -0.5756, -0.7268,  0.4939, -0.1708, -0.0659,
         -1.0750,  0.3750, -0.1872, -0.1182,  0.1482,  0.3745, -0.4823,  0.3275,
         -1.6070,  0.4665,  0.1964,  0.5482,  2.9102,  1.8901,  0.7657, -0.1239,
         -0.1513, -0.1591,  0.1554, -0.5196,  0.0599, -0.4722,  0.0295,  1.9088,
          0.2350, -0.2640,  0.4710, -0.0636,  0.1100, -0.0521, -0.2488,  0.1175,
         -0.2611, -0.1865,  0.0851, -0.1383,  0.2628,  0.4266, -0.0387,  0.4280,
         -0.3744,  0.1691, -0.9931,  0.0154, -0.5621, -0.1571,  0.2595, -0.1818,
         -0.5362,  0.4616,  0.1698,  0.0540,  0.1314,  0.4995, -0.2735,  0.3468,
          0.4329, -0.1187,  0.5122,  0.0418,  0.6882,  1.7881,  0.5487,  0.0822,
          0.8684,  0.9102,  0.3196, -0.1404,  0.7146, -2.3825, -0.4617,  0.0161,
          0.1518,  2.8468,  0.4177,  1.6036, -0.0228,  0.1325]],
       device='cuda:0')
result =a.flatten().tolist()
classlabel = [24, 29, 49, 38, 5, 3, 43, 48, 41, 37, 12, 31, 9, 42, 35, 28, 8, 14, 36, 4, 40, 19, 20, 21, 39, 53, 10, 25, 0, 44, 2, 17, 45]
        # index =self.current_Q.keys().item()
        # value = self.current_Q.values().item()
   
conditions =[]
for v in result:
   inx = result.index(v)
   if (inx>=(61)):
         conditions.append(v)
sorted_ = np.argsort(conditions)
for i in sorted_:
            co = env.conditions_reverse.get(classlabel[i])
            # print(co)
            pro_age = get_pro_age(co,env1.patient.age,env1.piors_map)/100
            pro_gender=get_pro_sex(co,env1.patient.gender,env1.piors_map)/100
            incidence = get_pro_incidence(co,env1.piors_map)
            conditions[i] = conditions[i]*pro_age*pro_gender+incidence
top_5 = []
sorted_ = np.argsort(conditions)
for i in range(5):
   top_5.append(classlabel[sorted_[-(i+1)]])
print(conditions[-1])
        

0.8531982848273515


In [ ]:
agent_test = rl.agent.MedAgent(
    env1,
    layer_config=layer_config,
    learning_start=learning_start,
    batch_size=batch_size,
    target_update=target_update,
    replay_capacity=replay_capacity,
    debug=False,
    train = False
)
# torch.save(agent.policy_network.state_dict(), cfg.model_path+'/dqn_checkpoint_policy.pth')
# agent_test.policy_network.load_state_dict(agent.target_network.state_dict())
# agent_test.policy_network.load_state_dict(agent.policy_network.state_dict())
# for target_param, param in zip(agent_test.policy_network.parameters(),agent.policy_network.parameters()): # 复制参数到目标网路targe_net
#             target_param.data.copy_(param.data)
# for target_param, param in zip(agent_test.policy_network.parameters(),agent.target_network.parameters()): # 复制参数到目标网路targe_net
#             target_param.data.copy_(param.data)
agent_test.eps_start = 0
agent_test.eps_end = 0
# print(cfg.model_path)
# agent_test.load(cfg.model_path)
agent_test.policy_network.load_state_dict(torch.load('./outputs/DQN/20211123-114930/models/dqn_checkpoint_policy.pth'))
testbench = rl.bench.MedBenchEval(agent_test,1000)
testbench.run_trial(debug=False)
rewards,success,average_rewards,average_steps =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn()
print(success)
print(average_rewards,average_steps)


In [19]:
test = "./data_preprossing/output/datas/min3/Asthma.csv"
env1 = rl.environment.RLBasicMedEnvNOPATIENT(
    data_file=test,
    symptom_map_file=symptom_map_file,
    condition_map_file=condition_map_file,
    symptom_nlice_file=symptom_nlice_file,
    body_parts=body_parts,
    excitation_enc=excitation_enc,
    frequency_enc=frequency_enc,
    nature_enc=nature_enc, 
    vas_enc=vas_enc, 
    onset_enc=onset_enc, 
    duration_enc=duration_enc,
    clf=clf,
    classifer="nb",
    max_turn=20,
    epoch = 1000
)


In [20]:
agent.env = env1
agent.train = False
agent.eps_start = 0
agent.eps_end = 0
agent.debug = False
testbench = rl.bench.MedBenchEval(agent,1000)
testbench.run_trial(debug=False)
rewards,success,average_rewards,average_steps,hint_rate =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn(),testbench.get_hint_rate()
print(success)
print(average_rewards,average_steps)
print(hint_rate)

0.0
-18.0 21.0
0.0


In [19]:
def find_symptom(index):
        symptoms=[]
        for s in index:
            if(s%8==0):
                symptoms.append(s/8)
        return symptoms


agent.env = env1
agent.train = False
agent.eps_start = 0
agent.eps_end = 0
agent.debug = False
testbench = rl.bench.MedBenchEval(agent,1)
testbench.run_trial(debug=False)
print(agent.env.patient)
print(agent.env.patient.condition)
print(np.where(agent.env.patient.symptoms==1)[0])
print(find_symptom(np.where(agent.env.patient.symptoms==1)[0]))
print(agent.env.inquiry_list)
rewards,success,average_rewards,average_steps =testbench.rewardList,testbench.get_success_rate(),testbench.get_average_rewards(),testbench.get_average_turn()
print(success)
print(average_rewards,average_steps)

AiMedPatient(symptoms=array([ 0,  0,  0,  0,  0,  0,  0,  0,  1, 11,  1,  1,  1,  1,  1,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,
   

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from matplotlib.font_manager import FontProperties

def plot_loss(loss,tag="train",env="milestone1",algo = "DQN"):
    sns.set()
    plt.title("average learning curve of {} for {}".format(algo,env))
    plt.xlabel('epsiodes')
    plt.plot(loss,label='loss')
    plt.legend()


def plot_rewards(rewards,tag="train",env="milestone1",algo = "DQN"):
    sns.set()
    plt.title("average learning curve of {} for {}".format(algo,env))
    plt.xlabel('epsiodes')
    plt.plot(rewards,label='reward')
    plt.legend()


In [ ]:
# plot_rewards(rewards)
eval = rewards
success = 0
plot_rewards(eval)
